In [2]:
import logging
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import requests
import sys
import time
from selenium.webdriver.common.keys import Keys
import urllib.parse as urlparse
from urllib.parse import parse_qs
import pandas as pd
sys.path.append('../src/')
from util.selenium_dispatcher import SeleniumDispatcher
from config import Config

In [3]:
 def get_request_token(kite_url):
        print('Starting to fetch request token for Kite API')
        selenium = SeleniumDispatcher(headless=True)
        driver = selenium.get_driver()
        driver.get(kite_url)
        time.sleep(4)
        username_field = driver.find_element_by_xpath("//input[@type='text']")
        username_field.send_keys(Config.KITE_USER_ID)
        password_field = driver.find_element_by_xpath("//input[@type='password']")
        password_field.send_keys(Config.KITE_PASSWORD)
        password_field.send_keys(Keys.ENTER)
        time.sleep(2)
        pin_field = driver.find_element_by_xpath("//input[@type='password']")
        pin_field.send_keys(Config.KITE_PIN)
        pin_field.send_keys(Keys.ENTER)
        time.sleep(2)
        url = driver.current_url
        parsed = urlparse.urlparse(url)
        token = parse_qs(parsed.query)['request_token'][0]
        print('Request token received!')
        selenium.destroy_driver()
        return token

In [4]:
kite = KiteConnect(api_key=Config.KITE_API_KEY)

In [5]:
print(kite.login_url())

https://kite.trade/connect/login?api_key=37wk1ehie3noonc1&v=3


In [6]:
data = kite.generate_session(get_request_token(kite.login_url()), api_secret=Config.KITE_API_SECRET)

Starting to fetch request token for Kite API
Request token received!


In [7]:
kite.set_access_token(data["access_token"])


In [10]:
instruments = kite.instruments(exchange='NSE')

In [11]:
for instrument in instruments:
    if instrument['tradingsymbol'] == 'AUROPHARMA':
        print(instrument['instrument_token'])

70401


In [17]:
required_instruments = list()

In [18]:
for instrument in instruments:
    if 'BANKNIFTY20312' in instrument['tradingsymbol']:
        required_instruments.append(instrument)

In [20]:
len(required_instruments)

210

In [23]:
df = pd.DataFrame.from_dict(required_instruments)

In [31]:
df_put = df[df.instrument_type == 'PE'].reset_index().drop('index', axis=1)

In [32]:
df_call = df[df.instrument_type == 'CE'].reset_index().drop('index', axis=1)

In [12]:
# Initialise
kws = KiteTicker(Config.KITE_API_KEY, data['access_token'])
tick_count = 0

In [13]:
def on_ticks(ws, ticks):
    # Callback to receive ticks.
    global tick_count
    print("Ticks: {}".format(ticks))
    tick_count = tick_count + 1
    print("Length: {}".format(tick_count))
#     if tick_count > 5:
#         ws.close()

def on_connect(ws, response):
    pass
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe([70401])
#     ws.subscribe([134657])
    # Set RELIANCE to tick in `full` mode.
#     ws.set_mode(ws.MODE_LTP, [134657, 408065, 237569, 389377, 2791681])
    ws.set_mode(ws.MODE_FULL, [70401])

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

In [14]:
# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

In [15]:
# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 70401, 'last_price': 647.7, 'last_quantity': 5, 'average_price': 659.36, 'volume': 28222329, 'buy_quantity': 395041, 'sell_quantity': 993126, 'ohlc': {'open': 650.0, 'high': 684.05, 'low': 631.25, 'close': 644.4}, 'change': 0.5121042830540143, 'last_trade_time': datetime.datetime(2020, 4, 22, 14, 41, 25), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2020, 4, 22, 14, 41, 25), 'depth': {'buy': [{'quantity': 57, 'price': 647.6, 'orders': 2}, {'quantity': 121, 'price': 647.5, 'orders': 2}, {'quantity': 100, 'price': 647.45, 'orders': 1}, {'quantity': 1248, 'price': 647.35, 'orders': 5}, {'quantity': 177, 'price': 647.3, 'orders': 4}], 'sell': [{'quantity': 495, 'price': 647.7, 'orders': 1}, {'quantity': 563, 'price': 647.75, 'orders': 1}, {'quantity': 794, 'price': 647.8, 'orders': 2}, {'quantity': 84, 'price': 647.85, 'orders': 2}, {'quantity': 70, 'price': 647.9, 'orders': 2}]}}]
Length: 1
Ticks:

Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Unhandled Error
Traceback (most recent call last):
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/defer.py", line 311, in addCallbacks
    self._runCallbacks()
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/base.py", line 447, in _continueFiring
    callable(*args, **kwargs)
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/base.py", line 706, in disconnectAll
    failure.Failure(main.CONNECTION_LOS